In [3]:
import numpy as np
import pandas as pd
import os
import glob


In [7]:
data_path = "data/singh_cp_pipeline_singlecell_images"

folder_paths = glob.glob(os.path.join(data_path, "*"))

len(folder_paths)

folder_paths

['data/singh_cp_pipeline_singlecell_images\\B02_s1_w16F89C55C-7808-4136-82E4-E066F8E3CB10',
 'data/singh_cp_pipeline_singlecell_images\\B02_s1_w1B1A7ADEA-8896-4C7D-8C63-663265374B72',
 'data/singh_cp_pipeline_singlecell_images\\B02_s1_w1D9CCD4E7-65E8-4426-9178-A955453164C5',
 'data/singh_cp_pipeline_singlecell_images\\B02_s1_w1E35EFE33-AB1C-4DB4-996B-D224E67F5C72',
 'data/singh_cp_pipeline_singlecell_images\\B02_s1_w1EA42BBDC-A8FA-44AB-A9B7-DA330F81E2A2',
 'data/singh_cp_pipeline_singlecell_images\\B02_s1_w1FDD9356D-FF65-44C5-8B07-6BEC430F6014',
 'data/singh_cp_pipeline_singlecell_images\\B02_s2_w125CF833D-7519-4E5D-A1A5-2A12E42E41E7',
 'data/singh_cp_pipeline_singlecell_images\\B02_s2_w156A83C2E-286E-4F75-A543-66F74D9B871C',
 'data/singh_cp_pipeline_singlecell_images\\B02_s2_w1862F353D-3553-4582-8FA5-7ED3FDC48FA1',
 'data/singh_cp_pipeline_singlecell_images\\B02_s2_w18894A89B-3736-4237-B423-1B5155F90BB0',
 'data/singh_cp_pipeline_singlecell_images\\B02_s2_w1E728D3CA-2152-49A6-AD0E-B24